In [172]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LinearRegression

In [173]:
test1 = pd.read_csv('./SE_rents2018_test1.csv')
train1 = pd.read_csv('./SE_rents2018_train-Copy1.csv')

In [174]:
df = train1.append(test1)
len(df)

14000

In [175]:
df.head()

,rental_id,addr_unit,building_id,bedrooms,bathrooms,size_sqft,created_at,addr_street,addr_city,addr_zip,...,has_garden,has_childrens_playroom,rent,no_fee,description,neighborhood,borough,unit,floornumber,line
0,7236931,#22A,551248,4,3.0,1300.0,2018-06-04 16:31:06,645 OCEAN AVENUE,Brooklyn,11226,...,0,0,3925,1,MAGNIFICENT GUT RENOVATED 4 BEDROOM 3 BATHROOM...,Flatbush,Brooklyn,22A,NaN,NaN
1,7331920,#406,3373,0,1.0,475.0,2018-07-03 20:11:43,93 WORTH STREET,New York,10013,...,0,0,3350,0,Want to live in the hottest neighborhood in to...,Tribeca,Manhattan,406,4.0,06
2,7139224,#18A,70747,3,3.0,1886.0,2018-05-02 16:10:27,305 EAST 51 STREET,New York,10022,...,1,0,11500,0,"High floor totally renovated 3 bedrooms, 3 bat...",Midtown East,Manhattan,18A,18.0,A
3,7364827,#23E,42076,1,1.0,827.0,2018-07-13 16:06:55,70 WEST 37TH STREET,New York,10018,...,0,0,3772,1,**SHORT TERM LEASES AVAILABLE**\n\n****Photogr...,Midtown South,Manhattan,23E,23.0,E
4,7242151,#3B,319156,3,1.0,1200.0,2018-06-05 16:35:40,43 MAC DONOUGH STREET,Brooklyn,11216,...,0,0,2595,1,This is a Huuge 3 bedroom apt. located in a pr...,Bedford-Stuyvesant,Brooklyn,3B,3.0,B


In [176]:
len(df)

14000

In [177]:
null_vals = df.apply(lambda col: sum(col.isnull()))
null_vals[null_vals > 0]

addr_unit         110
bin                 1
year_built        470
min_to_subway     143
description        16
neighborhood        3
unit              114
floornumber      1502
line             4135
dtype: int64

## Average Minutes to Subway

In [178]:
# Find the minutes to subway on average for each zipcode 

avg_subway_time_per_zip = df[['addr_zip', 'min_to_subway']].groupby('addr_zip').mean()
avg_subway_time_per_zip

,min_to_subway
addr_zip,
10001,2.679276
10002,2.547199
10003,2.291036
10004,3.406370
10005,1.524446
...,...
11435,4.737484
11691,6.105567
11692,7.033300


In [179]:
# Fill in missing min_to_subway using average min_to_subway for corresponding zipcode

df['min_to_subway'].fillna(df[df['min_to_subway'].isnull()].apply(lambda row: avg_subway_time_per_zip.loc[row['addr_zip']][0], axis=1), inplace=True)

## Floor Count

In [180]:
df.dropna(subset=['unit'], inplace = True)

In [181]:
# Setting up data to be able to model floor count
df = pd.get_dummies(df, columns=['borough'])
data = df[['floor_count','floornumber', 'has_childrens_playroom', 'has_pool', 'has_concierge', 'has_garage', 'has_gym', 'borough_Queens', 'borough_Bronx', 'borough_Manhattan', 'borough_Brooklyn', 'borough_Staten Island']]
data = data.dropna(subset=['floor_count', 'floornumber'])
data = data.reset_index()

In [185]:
# Fit the model

clf = LinearRegression()
features = ['floornumber', 'has_childrens_playroom', 'has_pool', 'has_concierge', 'has_garage', 'has_gym', 'borough_Queens', 'borough_Bronx', 'borough_Manhattan', 'borough_Brooklyn', 'borough_Staten Island']
clf.fit(data[features], data['floor_count'])

In [187]:
# Anything with floor count of 0 is labeled as NaN
df.loc[df['floor_count'] == 0, 'floor_count'] = np.NaN

In [188]:
test = df.loc[(df['floor_count'].isna()) & (df['floornumber'].isna() == 0)]

In [190]:
# Predict with the model and round up to get floor count 
imputed_floorcount = np.ceil(clf.predict(test[features]))

In [191]:
# Imput missing floor counts with predicted values
df.loc[(df['floor_count'].isna()) & (df['floornumber'].isna() == 0), 'floor_count'] = imputed_floorcount

In [192]:
# Manually fill in floor count for buildings where we don't have the floornumber or the floorcount 
df.loc[df['floor_count'].isna(), 'floor_count'] = [3.0, 3.0, 4.0, 7.0, 6.0, 2.0, 5.0, 6.0, 7.0, 6.0, 6.0, 18.0, 2.0, 4.0, 3.0, 12.0]

In [193]:
# Make sure that the maximum floor count is at least the maximum floornumber for the given building
df['floor_count'] = df[['floor_count', 'floornumber']].apply(lambda row: max(row['floor_count'], row['floornumber']), axis=1)

## Floornumber

In [194]:
df['unit_split'] = df['unit'].apply(lambda row: re.findall(r"[^\W\d_]+|\d+", row))

In [195]:
df['unit_num'] = df['unit_split'].apply(lambda row: list(filter(lambda x: x.isnumeric(), row)))

In [196]:
df['unit_str'] = df['unit_split'].apply(lambda row: list(filter(lambda x: x.isnumeric() == 0, row)))

In [198]:
# Make the floornumber the first number from the unit address

floornumber_fill = df.loc[(df['floornumber'].isna()) & (df['unit_num'].apply(lambda row: len(row) > 0))].apply(
    lambda row: int(row['unit_num'][0][0]) 
    if int(row['unit_num'][0][0]) <= row['floor_count'] 
    else np.NaN, axis = 1).dropna()

floornumber_fill = floornumber_fill.loc[~floornumber_fill.index.duplicated(keep='first')]
df['floornumber'].fillna(floornumber_fill, inplace=True)

In [200]:
# A = first floor, B = second floor, C = third ...

floornumber_fill = df.loc[(df['unit_num'].apply(lambda row: len(row) == 0)) & 
   (df['floornumber'].isna()) & 
   (df['unit_str'].apply(lambda row: len(row) > 0 and len(row[0]) == 1))].apply(
        lambda row: float(ord(row['unit_str'][0]) - 64)
        if float(ord(row['unit_str'][0]) - 64) <= row['floor_count']
        else np.NaN, axis = 1
).dropna()

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [202]:
# if address contains TOP, make the floornumber top floor

floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*TOP.*", row)))].apply(
    lambda row: row['floor_count'], axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [204]:
# if address contains UP, make the floornumber top floor

floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*UP.*", row)))].apply(
    lambda row: row['floor_count'], axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [205]:
# if address contains GN -> Garden or ground -> floorcount = 1
floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*G.*.*N.*", row)))].apply(
    lambda row: 1, axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [206]:
# if address contains GD -> Garden or ground -> floorcount = 1, contains some things missed before due to misspelled words

floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*G.*.D.*", row)))].apply(
    lambda row: 1, axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [207]:
# Penthouse
floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*PENT.*", row)))].apply(
    lambda row: row['floor_count'] , axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [208]:
# PH = Penthouse
floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*PH.*", row)))].apply(
    lambda row: row['floor_count'] , axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [209]:
floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*ONE.*", row)))].apply(
    lambda row: 1 , axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [210]:
floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*TWO.*", row)))].apply(
    lambda row: 2 , axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [211]:
floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*THREE.*", row)))].apply(
    lambda row: 3 , axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [212]:
df.loc[df['addr_unit'] == 'WALK-IN', 'floornumber'] = 1

In [213]:
# Parlor 
floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*PA.*L.*R.*", row)))].apply(
    lambda row: 1, axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [214]:
# Lower Level
floornumber_fill = df.loc[df['addr_unit'].apply(lambda row: bool(re.match(r".*LL.*", row)))].apply(
    lambda row: 1, axis = 1
)

df['floornumber'].fillna(floornumber_fill, inplace=True)

In [216]:
# If we didn't get the answer from any previous conditions, make floornumber top floor

floornumber_fill = df.loc[df['floornumber'].isna()].apply(lambda row: row['floor_count'], axis=1)
floornumber_fill = floornumber_fill.loc[~floornumber_fill.index.duplicated(keep='first')]
df['floornumber'].fillna(floornumber_fill, inplace=True)